In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from collections import defaultdict
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from preprocess import *

In [7]:
base_directory = os.getcwd()
data_directory = base_directory + "/../Data"
print("Base Directory:", base_directory)
print("Data Directory:", data_directory)

data_dirs = os.listdir(data_directory)

Base Directory: /Users/brad/Desktop/Plant_Pal/Modeling
Data Directory: /Users/brad/Desktop/Plant_Pal/Modeling/../Data


In [9]:
data = []

for data_dir in data_dirs:
    if data_dir == ".DS_Store":
        continue
    species, label = data_dir.split('___')  # how species and label are delineated in filenames
    image_files = os.listdir(os.path.join(data_directory, data_dir))
    for image in image_files:
        entry = [species, label, image]
        data.append(entry)

data_df = pd.DataFrame(data, columns=['Species', 'Condition', 'Image'])

In [8]:
# Split the data into train (80%) and temp (20%)
train_df, temp_df = train_test_split(data_df, test_size=0.2, stratify=data_df[['Species', 'Condition']], random_state=2)

# Split the temp data into validation (10%) and test (10%)
val_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df[['Species', 'Condition']], random_state=2)

print("Training set size:", len(train_df))
print("Validation set size:", len(val_df))
print("Test set size:", len(test_df))
# Adjust the training dataframe to ensure no more than 300 samples of any particular condition
train_df = train_df.groupby('Condition').apply(lambda x: x.sample(n=min(len(x), 300), random_state=2)).reset_index(drop=True)

print("Adjusted Training set size:", len(train_df))

Training set size: 77836
Validation set size: 9730
Test set size: 9730
Adjusted Training set size: 11052


In [9]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f'Using {device} for inference')
resnet50 = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_resnet50', pretrained=True)

Using cpu for inference


/Users/brad/env/lib/python3.10/site-packages/torch/hub.py:294: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/NVIDIA/DeepLearningExamples/zipball/torchhub" to /Users/brad/.cache/torch/hub/torchhub.zip
/Users/brad/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub/PyTorch/Classification/ConvNets/image_classification/models/common.py:13: UserWarning: pytorch_quantization module not found, quantization will not be availabl